### Retrieve Site and Parse HTML

In [5]:
#%pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/1"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(response.content, 'html.parser')

### Player Name and Team

In [6]:
t = soup.select('title')[0].text.strip()
player = t[:t.find(' - ')]

teams = {'Doosan': 'Doosan Bears', 
         'Hanwha': 'Hanwha Eagles', 
         'Kia': 'Kia Tigers', 
         'Kiwoom': 'Kiwoom Heroes', 
         'KT': 'KT Wiz', 
         'LG': 'LG Twins', 
         'Lotte': 'Lotte Giants', 
         'NC': 'NC Dinos', 
         'Samsung': 'Samsung Lions', 
         'SSG': 'SSG Landers'}

### Table Headers

In [7]:
# extract table header information
header = soup.select('thead th')
for i in range(len(header)):
  header[i] = header[i].text.strip()

# remove 'Game Stats' data
header = header[:header.index("Date")]
header = ['Name'] + header

### Get Table Contents 
Write to contents to dataframe, then write to CSV. 

In [ ]:
#%pip install pandas
import pandas as pd
from unicodedata import numeric


#to handle unicode data in table
def uni_to_num (unicode):
    if ('(' in unicode):
       return unicode
    elif len(unicode) == 0:
        return None
    elif len(unicode) == 1:
        num = numeric(unicode)
    elif unicode[-1].isdigit():
        # normal number, ending in [0-9]
        num = float(unicode)
    else:
        # Assume the last character is a vulgar fraction
        num = float(unicode[:-1]) + numeric(unicode[-1])
    return num


#data to be inserted into dataframe later
temp = []

# Parse row data and add to temp
rows = soup.select('tbody tr')

for r in rows:
  if (r.select_one('.left').text.strip() == 'Career'):
    break

  t = [player, r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]  
  for i in r.select('td')[2:]:
    t.append(uni_to_num(i.text.strip()))
  temp.append(t)


# init dataframe
data = pd.DataFrame(data = temp, columns = header)

# write to CSV
data.to_csv('data.csv')

# read CSV
#df = pd.read_csv('.csv')